In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.metrics import *

In [3]:
import os
os.chdir('/content/drive/MyDrive/데이콘/exercise_motion_classification/data/')

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [ ]:
! pip install catboost

In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

In [7]:
def quantile_75(x):
  return x.quantile(q=0.75)

def quantile_25(x):
  return x.quantile(q=0.25)

features = ['id','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
X_train = train[features].groupby('id').agg(['max','min','mean','median','std',quantile_75,quantile_25])
X_test = test[features].groupby('id').agg(['max','min','mean','median','std',quantile_75,quantile_25])

In [22]:
y_train = train_labels['label']

In [9]:
X_train.shape,y_train.shape

((3125, 42), (3125,))

In [10]:
X_train.reset_index(inplace=True)
X_train.drop(['id'],axis=1,inplace=True)

X_test.reset_index(inplace=True)
X_test.drop(['id'],axis=1,inplace=True)

In [ ]:
!pip install bayesian-optimization

In [13]:
from bayes_opt import BayesianOptimization

In [ ]:
base_model = CatBoostClassifier(verbose=True,classes_count =61)
base_model.fit(X_train,y_train)

In [15]:
X_test.head()

acc_x                      ...       gy_z                        
        max       min      mean  ...        std quantile_75 quantile_25
0 -0.275446 -1.564000 -1.018731  ...  12.251648    5.104833   -9.483683
1  0.627571 -1.929033 -0.522843  ...  61.604867   29.512729  -43.569149
2  2.972063 -0.792916  0.506947  ...  23.041463   12.942246  -13.266111
3  0.337281 -1.045889 -0.577603  ...  37.967372    8.768108   -9.005791
4  0.015642 -2.153047 -0.738640  ...  43.353007   28.747350  -12.864438

[5 rows x 42 columns]

In [16]:
y_pred = base_model.predict_proba(X_test)

In [18]:
y_pred

array([[1.83068539e-04, 1.20863884e-05, 2.62956886e-05, ...,
        6.48211526e-02, 1.01772491e-05, 3.37294775e-04],
       [2.87301284e-04, 9.58429023e-06, 2.89506439e-06, ...,
        4.69009329e-06, 1.24202912e-05, 2.26775393e-05],
       [1.48875332e-02, 3.92292342e-01, 4.33940468e-04, ...,
        1.95459779e-04, 6.01173816e-04, 3.76996658e-03],
       ...,
       [9.84316158e-04, 1.23636508e-05, 1.07268321e-05, ...,
        9.61152636e-06, 4.93649174e-06, 4.30575085e-04],
       [1.65404140e-04, 1.03057711e-03, 4.83056465e-05, ...,
        1.96976757e-05, 4.73936872e-06, 1.02388222e-05],
       [4.78468850e-05, 5.35222762e-06, 5.33261042e-06, ...,
        1.03764726e-05, 2.38064516e-06, 8.09326429e-05]])

In [19]:
submission.iloc[:,1:] = y_pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000183,0.000012,0.000026,0.000125,0.000167,0.000008,0.000069,0.000033,0.000029,0.005976,0.267482,0.084787,1.154093e-04,0.308335,0.007020,0.000273,0.000008,0.000034,6.081799e-06,0.000009,0.000027,0.000063,0.000085,0.000575,0.249536,0.000010,0.001683,2.289811e-05,1.834752e-05,1.854054e-05,0.000544,0.000420,0.000796,0.000016,0.000160,0.000117,0.002919,0.000044,0.000441,4.640199e-05,0.000014,0.000010,0.000260,0.000030,0.000024,4.076050e-05,0.000011,0.000031,0.000236,0.000113,0.000053,0.001072,2.774896e-04,0.000147,0.000040,0.000016,0.000031,0.000183,0.064821,0.000010,0.000337
1,3126,0.000287,0.000010,0.000003,0.000038,0.000066,0.000054,0.000005,0.000049,0.000009,0.000011,0.000024,0.000010,6.425692e-07,0.000010,0.000004,0.000077,0.000033,0.000002,5.681924e-06,0.000001,0.000043,0.000068,0.000086,0.000124,0.000005,0.000014,0.997864,2.132950e-05,9.079970e-07,6.780097e-07,0.000016,0.000006,0.000149,0.000018,0.000224,0.000088,0.000012,0.000005,0.000013,8.293639e-07,0.000070,0.000023,0.000032,0.000012,0.000004,2.350724e-06,0.000004,0.000041,0.000036,0.000028,0.000037,0.000052,1.439332e-06,0.000026,0.000017,0.000010,0.000023,0.000082,0.000005,0.000012,0.000023
2,3127,0.014888,0.392292,0.000434,0.002939,0.000121,0.000653,0.069739,0.006613,0.002433,0.000489,0.000738,0.000219,7.948029e-05,0.000392,0.000233,0.002335,0.009521,0.000521,2.116579e-04,0.000124,0.000205,0.000932,0.000242,0.000188,0.000322,0.000191,0.168940,1.921350e-02,6.203187e-04,5.193233e-04,0.000300,0.000168,0.007041,0.013818,0.007769,0.014636,0.004650,0.037671,0.001467,5.045938e-04,0.017368,0.000497,0.001478,0.002908,0.003542,1.374416e-01,0.000207,0.011692,0.013815,0.004790,0.000630,0.000367,3.292705e-04,0.000520,0.010734,0.001133,0.003405,0.000202,0.000195,0.000601,0.003770
3,3128,0.004840,0.000139,0.000582,0.000452,0.000036,0.000089,0.000023,0.000299,0.000067,0.000067,0.000357,0.000039,1.165893e-05,0.000092,0.000023,0.000400,0.000038,0.000048,1.526406e-05,0.000017,0.000052,0.001752,0.000035,0.000283,0.000390,0.000008,0.928124,8.480047e-05,5.653501e-05,9.877451e-05,0.000182,0.000126,0.003375,0.000752,0.000587,0.000234,0.000269,0.000042,0.000153,2.092713e-05,0.000030,0.000098,0.002691,0.001309,0.000171,2.823827e-05,0.000028,0.000048,0.002243,0.022162,0.004881,0.000075,3.584760e-05,0.000076,0.000089,0.000196,0.000032,0.000428,0.000089,0.000009,0.021020
4,3129,0.005347,0.000047,0.000017,0.000302,0.000363,0.000356,0.000015,0.000149,0.000146,0.000132,0.000011,0.000029,1.956567e-06,0.000026,0.000015,0.000350,0.000289,0.000027,8.652972e-05,0.000010,0.000061,0.000246,0.005010,0.000640,0.000013,0.000017,0.977908,6.620232e-06,1.351703e-05,3.415967e-06,0.000228,0.000064,0.000584,0.000279,0.003142,0.000126,0.000179,0.000015,0.000029,3.979291e-06,0.000023,0.000050,0.000113,0.000035,0.000012,4.086270e-06,0.000010,0.000016,0.000119,0.000777,0.001438,0.000019,3.546495e-06,0.000029,0.000009,0.000030,0.000013,0.000920,0.000010,0.000040,0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.001642,0.000012,0.000007,0.000163,0.011792,0.000190,0.000016,0.000079,0.000032,0.000924,0.000308,0.000313,4.614937e-06,0.000048,0.000049,0.000469,0.000028,0.000026,4.547219e-04,0.000018,0.000167,0.000048,0.000484,0.000057,0.000126,0.000058,0.977758,2.522917e-05,1.028912e-05,4.985447e-06,0.000374,0.000039,0.000502,0.000021,0.001408,0.000101,0.000184,0.000013,0.000039,5.068352e-06,0.000191,0.000041,0.000040,0.000029,0.000016,4.206501e-06,0.000072,0.000017,0.000631,0.000135,0.000052,0.000104,7.255817e-06,0.000324,0.000009,0.000016,0.000035,0.000164,0.000023,0.000026,0.000063
778,3903,0.000288,0.000009,0.000006,0.000

In [20]:
os.chdir('/content/drive/MyDrive/데이콘/exercise_motion_classification/submission/')
submission.to_csv('catboost_base_model.csv', index=False)

In [21]:
# from sklearn.model_selection import train_test_split

# x_train,y_train,x_valid,y_valid = train_test_split(X_train,y_train,test_size=0.3)

In [28]:
from catboost import Pool, cv

In [62]:
def catboost_eval(depth, learning_rate, min_data_in_leaf, l2_leaf_reg):
  params = {'depth': depth,
            'learning_rate': learning_rate,
            'min_data_in_leaf' : min_data_in_leaf,
            'l2_leaf_reg' : l2_leaf_reg,
            'iterations' : 100,
            'loss_function': 'MultiClass'}
  params['depth'] = int(round(depth))
  cv_dataset = Pool(data = X_train,label = y_train )
  cv_result = cv(cv_dataset, params, fold_count=3)
  return -1.0 * cv_result['train-MultiClass-std'].iloc[-1]

In [ ]:
cat_bo = BayesianOptimization(catboost_eval, {'depth':(2,10),
                                              'learning_rate':(0.001,0.2),
                                              'min_data_in_leaf':(1,5),
                                              'l2_leaf_reg':(10,50)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
cat_bo.maximize(init_points=3, n_iter=5, acq='ei')

|   iter    |  target   |   depth   | l2_lea... | learni... | min_da... |
-------------------------------------------------------------------------
0:	learn: 3.9466845	test: 3.9484522	best: 3.9484522 (0)	total: 228ms	remaining: 22.5s
1:	learn: 3.8544818	test: 3.8565492	best: 3.8565492 (1)	total: 450ms	remaining: 22.1s
2:	learn: 3.7410682	test: 3.7431862	best: 3.7431862 (2)	total: 682ms	remaining: 22.1s
3:	learn: 3.6468103	test: 3.6489364	best: 3.6489364 (3)	total: 910ms	remaining: 21.8s
4:	learn: 3.5690976	test: 3.5712762	best: 3.5712762 (4)	total: 1.13s	remaining: 21.5s
5:	learn: 3.4942310	test: 3.4972102	best: 3.4972102 (5)	total: 1.36s	remaining: 21.4s
6:	learn: 3.4262407	test: 3.4293148	best: 3.4293148 (6)	total: 1.58s	remaining: 21s
7:	learn: 3.3710664	test: 3.3741178	best: 3.3741178 (7)	total: 1.8s	remaining: 20.8s
8:	learn: 3.3195005	test: 3.3226001	best: 3.3226001 (8)	total: 2.03s	remaining: 20.6s
9:	learn: 3.2798374	test: 3.2831442	best: 3.2831442 (9)	total: 2.25s	remaining: 2